# Avaliando landmarks - ResNet18 - GridSearch(epoch vs batch size)

## Import data

In [1]:
# !pip install seaborn --user
# !pip install tensorflow
# !pip install image-classifiers

In [2]:
import os
import tensorflow as tf
from keras import callbacks, Model
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras import optimizers
from keras.utils.io_utils import HDF5Matrix
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, Activation, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import random
import seaborn as sns
from sklearn import datasets, metrics

%matplotlib inline

/opt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11494465003015386130
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 13355929295398196926
physical_device_desc: "device: XLA_CPU device"
]


## Classificando Landmarks (Análise dos dados)

### Lendo o conjunto de dados

In [55]:
seed = random.seed(42)

sample_datagen = ImageDataGenerator(rescale=1./255)
base_path = '/home/ivancosta/education/mdc-tcc/Landmarks/data/landmarks'
target_size = (224, 224)
input_shape = (224, 224, 3)
classes = ["47378", "120885", "85758", "180901", "48522", "101399", 
           "190822", "97734", "146250", "186080", "21253", "142644", 
           "31531", "165596", "56827", "38482", "20102", "178519", 
           "152827", "173511"]

seed = 7
np.random.seed(seed)
sample_generator = sample_datagen.flow_from_directory(base_path + '/subset_train',
                                                      target_size=target_size,
                                                      batch_size=32,
                                                      class_mode="sparse",
                                                      seed = seed)

sample_test_generator = sample_datagen.flow_from_directory(base_path + "/subset_test",
                                                           target_size = target_size,
                                                           batch_size = 32,
                                                           class_mode = "categorical",
                                                           seed = seed)

total_classes = np.max(sample_generator.labels) + 1

x_sample, y_sample = sample_generator.next()
x_sample_test, y_sample_test = sample_test_generator.next()
print('\n')
print('Showing y sample:', y_sample)
print('\n')
print('samples in train: %i' % sample_generator.labels.shape,
      'samples in test: %i' % sample_test_generator.labels.shape,
      'features: %s' % str(x_sample.shape[1:]),
      'classes: %i' % total_classes,
      sep='\n', end='\n\n')

print('shape:', x_sample.shape, x_sample_test.shape)

Found 12508 images belonging to 20 classes.
Found 3128 images belonging to 20 classes.


Showing y sample: [ 8.  6.  8.  3.  8. 16. 18.  8.  2. 19.  3.  7.  1. 16. 11. 15. 17.  5.
  2. 14.  5. 15.  1. 19. 15.  2.  7.  4.  9. 18.  8.  2.]


samples in train: 12508
samples in test: 3128
features: (224, 224, 3)
classes: 20

shape: (32, 224, 224, 3) (32, 224, 224, 3)


## Treinamento 
### Parâmetros para treinamento e validação

In [5]:
rms = optimizers.RMSprop(lr = 0.0002,
                         decay = 1e-6)

device = '/gpu:0'

epochs = 64
batch = 32

Instructions for updating:
Colocations handled automatically by placer.


## Funções de auxílio

In [56]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, TerminateOnNaN, ReduceLROnPlateau
from keras.wrappers.scikit_learn import KerasClassifier

import types

## Adapted from https://stackoverflow.com/questions/47279677/how-use-grid-search-with-fit-generator-in-keras
class KerasBatchClassifier(KerasClassifier):

    def fit(self, X, y=None, **kwargs):

        # taken from keras.wrappers.scikit_learn.KerasClassifier.fit ###################################################
        if self.build_fn is None:
            self.model = self.__call__(**self.filter_sk_params(self.__call__))
        elif not isinstance(self.build_fn, types.FunctionType) and not isinstance(self.build_fn, types.MethodType):
            self.model = self.build_fn(**self.filter_sk_params(self.build_fn.__call__))
        else:
            self.model = self.build_fn(**self.filter_sk_params(self.build_fn))

        loss_name = self.model.loss
        if hasattr(loss_name, '__name__'):
            loss_name = loss_name.__name__

        ################################################################################################################
        epochs = self.sk_params['epochs'] if 'epochs' in self.sk_params else 100
        batch = self.sk_params['batch_size'] if 'batch_size' in self.sk_params else 32
        print('epochs=%s', epochs)
        print('batch=%s', batch)
        
        patience = epochs // 3
        
        base_path = kwargs['base_path']
        target_size = kwargs['target_size']
        
        datagen = ImageDataGenerator(rescale = 1./255, validation_split = 0.2)

        validation_flow = datagen.flow_from_directory(
            base_path + "/subset_train",
            target_size = target_size,
            batch_size = batch,
            class_mode = "categorical",
            subset ='validation')
        
        validation_steps = validation_flow.samples // batch
        
        train_flow = datagen.flow_from_directory(
            base_path + "/subset_train",
            target_size = target_size,
            batch_size = batch,
            class_mode = "categorical",
            subset = 'training')
        
        train_steps = train_flow.samples // batch

        early_stopping = EarlyStopping(patience=patience, 
                                       restore_best_weights = True,
                                       verbose=5, 
                                       mode="auto")
        model_checkpoint = ModelCheckpoint("results/best_weights.{epoch:02d}-{loss:.5f}.hdf5", 
                                           verbose=1, 
                                           save_best_only=True, 
                                           mode="auto")
        terminate_onnan = TerminateOnNaN()
        reduce_plateau = ReduceLROnPlateau(patience=patience)
        
        callbacks = [early_stopping, model_checkpoint, terminate_onnan, reduce_plateau]

        self.__history = self.model.fit_generator(
            train_flow,  
            steps_per_epoch=train_steps,
            validation_data=validation_flow, 
            validation_steps=validation_steps, 
            epochs=epochs,
            callbacks=callbacks,
            verbose = 1
        )

        return self.__history

    def score(self, X, y, **kwargs):
        kwargs = self.filter_sk_params(Sequential.evaluate, kwargs)

        loss_name = self.model.loss
        if hasattr(loss_name, '__name__'):
            loss_name = loss_name.__name__
        if loss_name == 'categorical_crossentropy' and len(y.shape) != 2:
            y = to_categorical(y)
        outputs = self.model.evaluate(X, y, **kwargs)
        if type(outputs) is not list:
            outputs = [outputs]
        for name, output in zip(self.model.metrics_names, outputs):
            if name == 'acc':
                return output
        raise Exception('The model is not configured to compute accuracy. '
                        'You should pass `metrics=["accuracy"]` to '
                        'the `model.compile()` method.')

    @property
    def history(self):
        return self.__history

In [58]:
from sklearn.model_selection import GridSearchCV

def grid_search(create_model, param_grid, train_epochs = epochs):
    model = KerasBatchClassifier(build_fn=create_model)

    grid = GridSearchCV(estimator=model, param_grid=param_grid)
    with tf.device(device):
        return grid.fit((1, 1, 1), base_path = base_path, target_size = target_size, n_jobs=-1)

### Definindo a rede

In [15]:
from classification_models.resnet import ResNet18, preprocess_input

def build_resNet18(optimizer = rms):
    model = ResNet18(input_shape = input_shape,
                   weights = "imagenet",
                   include_top=False)

    for layer in model.layers:
          layer.trainable = False

    output = model.output

    output = Flatten(name = 'flat_mdc')(output)

    output = Dense(total_classes,
                   activation ='softmax',
                   name = 'saida_mdc')(output)

    model = Model(inputs = model.input, outputs = output)

    model.compile(loss ='categorical_crossentropy', 
                  optimizer = optimizer, 
                  metrics=['accuracy'])

    model.summary()
    return model

In [16]:
def build_resNet18_tuning():
    model = build_resNet18()
    for layer in model.layers:
        layer.trainable = True

    model.compile(loss ='categorical_crossentropy',
                  optimizer = rms,
                  metrics=['accuracy'])

    model.summary()
    return model

### GridSearch 1 - batch size vs epochs

In [59]:
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]

param_grid = dict(batch_size=batch_size, epochs=epochs)
grid_search(build_resNet18_tuning, param_grid)

/home/ivancosta/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 224, 224, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_181 (ZeroPadding (None, 230, 230, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 112, 112, 64) 9408        zero_padding2d_181[0][0]         
__________________________________________________________________________________________________
bn0 (Batch

Found 2492 images belonging to 20 classes.
Found 10016 images belonging to 20 classes.
Epoch 1/10
  95/1001 [=>............................] - ETA: 44:08 - loss: 4.2988 - acc: 0.2884

KeyboardInterrupt: 

In [ ]:
learning_curve_model(history_resNet18_tuning)

### Avaliando modelo treinado

In [ ]:
# evaluate_model(resNet18_tunning_model, 'ResNet18 - Tuning')